<a href="https://colab.research.google.com/github/seongwoojang1123/Exploring-Key-Determinants-of-Halitosis-Through-a-Data-Analysis/blob/main/1_Halitosis_data_preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import zscore

In [26]:
file_path = '/content/drive/MyDrive/Halitosis/20241107 20231015 data_xerostomia halitosis 11111111.xls'

df = pd.read_excel(file_path)


# 평균값으로 채울 컬럼
mean_fill_columns = ['pH', 'BufferCapacity']

# 0으로 채울 컬럼
zero_fill_columns = ['S_Hypertension', 'S_DM', 'S_Hyperlipidemia', 'S_RenalDiseases', 'S_LiverDiseases',
                     'S_Rheumatism', 'S_Osteoporosis', 'S_CVD', 'S_TD', 'S_MentalDisorders', 'S_UrinaryDiseases','S_Asthma']

# 1. 평균값으로 결측치 채우기
for column in mean_fill_columns:
    df[column] = df[column].fillna(df[column].mean())

# 2. 0으로 결측치 채우기
df[zero_fill_columns] = df[zero_fill_columns].fillna(0)

# 3. 빈 값 (공백, NaN 등)을 0으로 채우기
df = df.replace(r'^\s*$', 0, regex=True)  # 공백 값을 0으로 변환
df = df.fillna(0)  # NaN 값을 0으로 변환

columns = df.columns
print(len(columns))
print(columns)
df.count()

df.to_excel('/content/drive/MyDrive/Halitosis/241121_Halitosis_data preprocessing.xlsx', index = False)

49
Index(['Halitosis_subjective', 'Sex', 'Age', 'Elderly', 'Super_Elderly',
       'Xerostomia_subjective', 'UFR', 'SFR', 'pH', 'BufferCapacity', 'VAS',
       'StickySaliva', 'Oralhygiene', 'Calculus', 'O_Stomatitis', 'O_RAU',
       'O_Candidiasis', 'O_Periodontitis', 'O_LichenPlanus', 'O_Sialodochitis',
       'O_Glossodynia', 'O_BMS', 'S_Hypertension', 'S_DM', 'S_Hyperlipidemia',
       'S_RenalDiseases', 'S_LiverDiseases', 'S_Rheumatism', 'S_Osteoporosis',
       'S_CVD', 'S_TD', 'S_MentalDisorders', 'S_UrinaryDiseases', 'S_Asthma',
       'S_CancerOp', 'NumberofSystmicDiseases', 'M_Hypertension', 'M_DM',
       'M_Osteoporosis', 'M_Hyperlipidemia', 'M_CV', 'M_TD', 'M_GI',
       'M_UrinaryDiseases', 'M_Arthritis', 'M_Rheumatism', 'M_Anxiolytic',
       'M_SleepingPills', 'M_Aspirin'],
      dtype='object')


,0
Halitosis_subjective,821
Sex,821
Age,821
Elderly,821
Super_Elderly,821
Xerostomia_subjective,821
UFR,821
SFR,821
pH,821
BufferCapacity,821


In [25]:
file_path = '/content/drive/MyDrive/Halitosis/241121_Halitosis_data preprocessing.xlsx'

df = pd.read_excel(file_path)

def count_outliers_by_zscore(df, z_threshold=10, min_unique=3):
    """
    df           : 판별할 DataFrame
    z_threshold  : abs(z-score)가 이 값을 초과하면 이상치로 판정 (기본 ±10)
    min_unique   : 해당 컬럼의 고윳값 개수가 이 값보다 작으면(즉, 0/1처럼 범주가 매우 적으면) 제외
    """
    outlier_counts = {}

    # 1) 숫자형 컬럼만 추출
    numeric_cols = df.select_dtypes(include=['float64', 'int64']).columns

    # 2) 고윳값 개수가 너무 적은(이진 등) 컬럼은 제외
    numeric_cols = [col for col in numeric_cols if df[col].nunique() >= min_unique]

    # 3) 각 컬럼별로 Z-score를 이용해 이상치 개수 계산
    for column in numeric_cols:
        # 결측치 제거 후 Z-score 계산
        valid_series = df[column].dropna()
        z_scores = zscore(valid_series)

        # z_threshold 초과하는 값들의 개수
        outlier_count = (np.abs(z_scores) > z_threshold).sum()

        outlier_counts[column] = outlier_count

    return outlier_counts


outlier_dict = count_outliers_by_zscore(df, z_threshold=10)

# 컬럼별 이상치 개수 출력
for col, cnt in outlier_dict.items():
    print(f"'{col}' 컬럼의 이상치 개수: {cnt}")

total_outliers = sum(outlier_dict.values())
print(f"전체 이상치 개수(중복 합산): {total_outliers}")

'Age' 컬럼의 이상치 개수: 0
'Super_Elderly' 컬럼의 이상치 개수: 0
'UFR' 컬럼의 이상치 개수: 0
'SFR' 컬럼의 이상치 개수: 1
'pH' 컬럼의 이상치 개수: 0
'BufferCapacity' 컬럼의 이상치 개수: 0
'VAS' 컬럼의 이상치 개수: 0
'Oralhygiene' 컬럼의 이상치 개수: 0
'Calculus' 컬럼의 이상치 개수: 0
'NumberofSystmicDiseases' 컬럼의 이상치 개수: 0
전체 이상치 개수(중복 합산): 1
